Note: I have forgotten to check for f3 from f2, that f3("empty")+f("1...n")=-2.

At the end 

Before outputtin to txt files I have removed those [f2,f3,dim_poly] where the f3 fails the above condition.


# Overview

For Pn Msa cases.

We construct the convex polytopal region denoted $R_f$ which are intersections of  $$b\le Ax \le b+1.$$

We determine the number of cases where $R_f $ are of dimension less that the top dimensios and select the $f$ for which this occurs. The functions correspond to those for which there exists non universal $\phi$.

For the genus 2 case we follow the documentation to determine which f2,f3 pairs are degnerate for n=3,4 and tranlsated case.

# Investigate (f2,f3) for P3-P4 and translate cases 

## Functions

In [77]:
def load_l_f2f3(path):

    """
    In:
    out:
    Obj:Load:list of info
    """
    #load
    with open(path, 'rb') as file:
        loaded_df = pickle.load(file)

    return loaded_df

#remember sage doesnt have pandas so for each (f2,f3) need to determine hyperplane inequalities.

In [78]:
def get_hyperplanes_f2(f2): #put into format of sage b<Ax<b+1
    
    """
        In: f2 msa function

        Obj:return a list of equalities to define the polytope for f2

        Out:list of equality in correct format for hyperplanes for sage

        First get inputs then get outcomes.
        input:[0, 1]
        outcome:[0, 1, 0, 0]

        input:[0, 2]
        outcome:[0, 0, 1, 0]

        input:[2, 3]
        outcome:[2, 0, 0, 1]
    """
    
    #init
    #Leaving this, this way because im stupid 
    
    dataframe = pd.DataFrame([f2])
    #f2={'1': 0, '2': 0, '3': 0, '12': 1, '13': 1, '23': 0, '123': 1}
    dataframe_list = dataframe.values.tolist()
    #[[0, 0, 0, 1, 1, 0, 1]]
    df_temp_col_list = dataframe.columns.tolist()
    #['1', '2', '3', '12', '13', '23', '123']
    df_col_list =[ list(x) for x in df_temp_col_list]
    #[['1'], ['2'], ['3'], ['1', '2'], ['1', '3'], ['2', '3'], ['1', '2', '3']]

    for i in dataframe_list: #creates row at a time #Here its of size 1

        #prep lists with constants
        l_i= [ [x,] for x in i]
        # print("l_i",l_i)
        
        """
        Example 
        Suppose we wish to write 
        
        for
        b<x+y<b+1
        
        The inequalities we want are.
        res_lower_example=[-b,1,1]
        res_upper_example=[b+1,-1,-1]
        
        Outputs:
        res_lower [[0, 1], [0, 2], [0, 3], [0, 1, 2], [-1, 1, 3], [-1, 2, 3], [-1, 1, 2, 3]]
        res_upper [[1, -1], [1, -2], [1, -3], [1, -1, -2], [2, -1, -3], [2, -2, -3], [2, -1, -2, -3]]
        """
        
        #looks like [-b,1,1] for b \le x+y
        res_lower = [list(itertools.chain(*i)) for i in zip(l_i, df_col_list)] # Ax>b #todo need to insert minus for b term. for Ax-b>0
        for j in range(len(res_lower)):
            res_lower[j]=[ int(x) for x in res_lower[j]]#multpliy all elements by negative to change inequality direction.-Ax
            res_lower[j][0]=-int((res_lower[j][0])) # -b part.
        # print("res_lower",res_lower) 
        
        res_upper = [list(itertools.chain(*i)) for i in zip(l_i, df_col_list)]#Ab<b+1 need to arrange into same format as above -Ax+(b+1)>0
        for j in range(len(res_upper)):
            res_upper[j]=[ -1*int(x) for x in res_upper[j]]#multpliy all elements by negative to change inequality direction.-Ax
            res_upper[j][0]=-int((res_upper[j][0]))+1 # Todo Possible check for negative position
        # print("res_upper",res_upper) #look like [b+1,-1,-1] for x+y \le b+1
        
        res_combined= res_lower +res_upper
        
        #change format DONE!
        formatted_ResCombine=[]
        
        for item in res_combined:#res_combined = [[]...[]], item = []
            # input:[-1, 1, 3]
            # outcome:[-1, 1, 0, 1]
            
            # print(f"input:{item}")
            x=[0]*(max(len(l) for l in res_combined)) #finding [0,0,0,0,0] (getting this format) for P_4 ie n+1 
            # print(f"x:{x}")
            
            x[0]=item[0] #starts at b terms
            
            
            for k in range(1,len(item)):
                #for the x_i terms                
                # print("item[k]",item[k])
                if int(item[k])<0: #!!!!! Changed
                    x[-int(item[k])]=-1 #
                else:
                    x[int(item[k])]=1
            # print(f"outcome:{x}")
            
            formatted_ResCombine.append(x)   #terms looks like [[]...[]]
            
    return formatted_ResCombine #a list of the functions P_n

In [83]:
def get_comp(k,max_set):#for get_hyperplanes_f3
    #get complement 
    # number_string = "12345"
    # y = "134"
    z = "".join(char for char in max_set if char not in k)
    return z #"25"

def get_res_lower(values_list,col_list, col_list_comp):#for get_hyperplanes_f3
    
    """
    Here we do the 1+2f3(A)<x_A -x_A^c part for all A, this is recorded in res_lower
    
    and put it into the form, 0<-(1+2f3(A)) + x_A -x_A^c as a list # (STAR!)
    """
    
    #constant terms
    l_i=[[-(1+(2*x)),] for x in values_list]
    
    # Combine the lists element-wise using zip
    combined_lists = zip(l_i, col_list, col_list_comp)
    # print("combined_lists",list(combined_lists)) # combined_lists [(-3, ['1'], ['2', '3']), (-1, ['2'], ['1', '3']), (-1, ['3'], ['1', '2']), (-1, ['1', '2'], ['3']), (-1, ['1', '3'], ['2']), (1, ['2', '3'], ['1'])]    res_lower=[]
    
    
    res_lower=[]
    for constant, col_list, col_comp_list in combined_lists:
        formated_col_l=[int(x) for x in col_list] #see (STAR!)
        # col_comp_list = ['2', '3']
        form_col_comp_list=[-int(x) for x in col_comp_list] #see (STAR!)
        # Create the result list using list comprehension
        res_lower_term=constant + formated_col_l+form_col_comp_list
        # print(result)  # Output: [-2, 1, -2, -3]
        # print("res_lower_term",res_lower_term)
        res_lower.append(res_lower_term)
    return res_lower

def get_res_upper(values_list,col_list, col_list_comp):#for get_hyperplanes_f3
    
    """
    Here we do the x_A -x_A^c<3+2f3(A) part for all A, this is recorded in res_lower 
    
    and put it into the form, 0<+(3+2f3(A)) - x_A +x_A^c as a list (STAR!)
    """
    
    """
        [-2, -1, -1, -1, -1, 0]
        res_lower_term [3, 1, -2, -3]
        res_lower_term [1, 2, -1, -3]
        res_lower_term [1, 3, -1, -2]
        res_lower_term [1, 1, 2, -3]
        res_lower_term [1, 1, 3, -2]
        res_lower_term [-1, 2, 3, -1]
        [-2, -1, -1, -1, -1, 0]
        res_upper_term [-1, -1, 2, 3]
        res_upper_term [1, -2, 1, 3]
        res_upper_term [1, -3, 1, 2]
        res_upper_term [1, -1, -2, 3]
        res_upper_term [1, -1, -3, 2]
        res_upper_term [3, -2, -3, 1]
    """
    
    #constant terms
    l_i=[[3+(2*x),] for x in values_list] #see (STAR!) 
    
    # Combine the lists element-wise using zip
    combined_lists = zip(l_i, col_list, col_list_comp)    
    
    res_upper=[]
    for constant, col_list, col_comp_list in combined_lists:
        formated_col_l=[-int(x) for x in col_list] #see (STAR!) 
        # col_comp_list = ['2', '3']
        form_col_comp_list=[int(x) for x in col_comp_list] #see (STAR!) 
        # Create the result list using list comprehension
        res_upper_term=constant + formated_col_l+form_col_comp_list
        # print("res_upper_term",res_upper_term)
        res_upper.append(res_upper_term)
    return res_upper

def reformat_hyperplanes(res_combined):#for get_hyperplanes_f3
    
    #change format DONE!
    
    formatted_ResCombine=[] #have earlier now
    for item in res_combined:#res_combined = [[]...[]], item = []
        # input:[-1, 1, 3]
        # outcome:[-1, 1, 0, 1]

        # print(f"input:{item}")
        x=[0]*(max(len(l) for l in res_combined)) #finding [0,0,0,0,0] (getting this format) for P_4 ie n+1 
        # print(f"x:{x}")

        x[0]=item[0] #starts at b terms


        for k in range(1,len(item)):
            #for the x_i terms                
            # print("item[k]",item[k])
            if int(item[k])<0: #!!!!! Changed
                x[-int(item[k])]=-1 #
            else:
                x[int(item[k])]=1
        # print(f"outcome:{x}")

        formatted_ResCombine.append(x)   #terms looks like [[]...[]]
    
    return formatted_ResCombine

def get_hyperplanes_f3(f3,n): #put into format of sage bottom<Ax<top
    
    """
        Obj:return a list of equalities to define the polytope for f2

        In: f3 from some msa f2

        Out:list of equality in correct format for hyperplanes for sage

        First get inputs then get outcomes.
        input:[0, 1]
        outcome:[0, 1, 0, 0]

        input:[0, 2]
        outcome:[0, 0, 1, 0]

        input:[2, 3]
        outcome:[2, 0, 0, 1]
    """
    
    #init
    number_list = [str(i) for i in range(1, n + 1)] #if n=3 want 123 so can tail ceiling nad det f3(empty) value
    max_set  = "".join(number_list)
    formatted_ResCombine=[]
    
    #Get res_upper/lower hyperplanes for max_set and empty
    empty_lower_term=[-(1+(2*f3["empty"])),]+[-int(x) for x in max_set]
    empty_upper_term=[3+(2*f3["empty"]),]+[int(x) for x in max_set]
    empty_hpy=[empty_lower_term,empty_upper_term]
        
    max_set_lower_term=[-(1+(2*f3[max_set])),]+[int(x) for x in max_set]
    max_set_upper_term=[3+(2*f3[max_set]),]+[-int(x) for x in max_set]
    max_set_hpy=[max_set_lower_term,max_set_upper_term]
    
    #We remove "empty" and handle seperately with max_set as complment
    f3_copy=dict(f3)
    del f3_copy[max_set]
    del f3_copy["empty"]
    values_list = list(f3_copy.values())
    keys_list = list(f3_copy.keys())
    col_list =[ list(x) for x in keys_list]
    col_list_comp=[list(get_comp(term,max_set)) for term in col_list]

    res_lower=get_res_lower(values_list,col_list, col_list_comp)
    res_upper=get_res_upper(values_list,col_list, col_list_comp)

    #combining
    res_combined= res_lower +res_upper + empty_hpy+max_set_hpy
    #into correct format
    
    # print("res_combined",res_combined)
    total_formatted_ResCombine=  reformat_hyperplanes(res_combined)

    # print("total_formatted_ResCombine",total_formatted_ResCombine)
    return total_formatted_ResCombine #a list of the functions P_n

In [92]:
def get_f2f3_poly_inequals(n,f2,f3):#for get_f2f3l_w_inequals
    
    """
    In:
    out:
    Obj:Get dim: Construct polytope for hyperplanes given by f2 and f3 
    """
    #get hyperplanes
    f2_hyp=get_hyperplanes_f2(f2)
    f3_hyp=get_hyperplanes_f3(f3,n)
    
    poly_inequals=f2_hyp+f3_hyp
    return [f2,f3,poly_inequals]

def get_f2f3l_w_inequals(n,data):#main

    """
    In:data is of the form [[f2,f3l],...]

    where each f2 is in Pn msa and f3l is the list of f3 for the given f2 see docs.

    out:output will be [[f2,[[f2,f3,inequals],...,[f2,f3,inequals]],...,[f2,[[f2,f3,inequals],...,[f2,f3,inequals]]]

    Obj:Main: for each f2 will have [f2,f3,poly_dim] for each f3
    """

    output=[]
    
    for f2_f3l_packet in data:    
        [f2,f3l]=f2_f3l_packet
            
        triple_l=[]
        for f3 in f3l:

            pair=[f2,f3]

            triple=get_f2f3_poly_inequals(n,*pair) #[f2,f3,inequals]

            triple_l.append(triple)

        f2_f3l_packet_w_inequal=[f2,triple_l] #format??
        output.append(f2_f3l_packet_w_inequal)

    return output

In [116]:
def get_f2f3l_w_dim_poly(n,data):
    
    """
    To be run in sage.
    
    Obj: for [[f2,[[f2,f3,inequals],...,[f2,f3,inequals]],...,[f2,[[f2,f3,inequals],...,[f2,f3,inequals]]] for each 
    [f2,[[f2,f3,inequals],...,[f2,f3,inequals]]
    """
    new_data=[]
    for f2_packet in data:        
        #We take [f2,[[f2,f3,inequals],...,[f2,f3,inequals]] and return [f2,[[f2,f3,dimpoly],...,[f2,f3,dimpoly]]
        #where dimpoly is the dim of the poly given by the inequals
        
        new_f2_packet=[f2_packet[0],]

        f2f3dim_ls=[] #[f2,]
        for triple in (f2_packet[1]):
#             triple of form [f2,f3,inequals]
            triple_w_polydim=get_f2f3_poly_dim(n,*triple)
            f2f3dim_ls.append(triple_w_polydim)
    
        new_f2_packet=new_f2_packet+f2f3dim_ls
        
#         print("new_f2_packet",new_f2_packet)
        
        new_data.append(new_f2_packet)
    
    return new_data

#we now save this, load it into sage and run polytope constuction

def get_f2f3_poly_dim(n,f2,f3,poly_inequals):#for get_f2f3l_w_dim_poly
    """
    In:
    out:
    Obj:Get dim: Construct polytope for hyperplanes given by f2 and f3 
    Obj:Given list of hyperplanes for f2 and f3 together
    we construct a the polytope and return its dimension
    """
    L_list_line=poly_inequals
    Poly=Polyhedron(ieqs=L_list_line)
    dim_p=dim(Poly)
    
    if dim_p <n:
        dim_p="Empty Dimensional"
    if dim_p ==n:
        dim_p=f"Top dimensional:{n}"
    return [f2,f3,dim_p]

## Cases 

We are now in the position to determine which (f2,f3) pairs have a degenerate polytope.

We record the results as a text file recording the pairs (f2,f3) and the dimension see:

"\MSA\Dimension_Analysis\results\f2_f3_txt_poly_dims"

The files of interest are: this is collated data taken from Pn_abridged i.e the data of (f2,f3) for f2 in Pn and associated f3.

path="\results\poly_inv\P3_f2_f3l.pkl"
path="\results\poly_inv\P3_trans_f2_f3l.pkl"

path="\results\poly_inv\P4_f2_f3l.pkl"
path="\results\poly_inv\P4_trans_f2_f3l.pkl"

path="\results\poly_inv\P5_degen_f2_f3l.pkl"

### P3 Done

In [98]:
path="results\poly_inv\P3_f2_f3l.pkl"
n=3

In [99]:
f2f3l=load_l_f2f3(path)
data=f2f3l

In [101]:
#determine all inequalts for f2 f3 and store as  [[f2,[[f2,f3,inequals],...,[f2,f3,inequals]],...,[f2,[[f2,f3,inequals],...,[f2,f3,inequals]]]
out=get_f2f3l_w_inequals(n,data)

In [102]:
# #save to pass to sage
# with open('results\hyperplanes_P3_f2f3.pkl', 'wb') as file:
#     pickle.dump(out, file)

In [113]:
# #load in sage
# with open('results\hyperplanes_P3_f2f3.pkl', 'rb') as file:
#     data = pickle.load(file)

<>:2: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<ipython-input-113-8a799c422c96>:2: DeprecationWarning: invalid escape sequence \h
  with open('results\hyperplanes_P3_f2f3.pkl', 'rb') as file:


In [118]:
n=3
new_data=get_f2f3l_w_dim_poly(n,data)

In [120]:
# #save from sage
# with open('results\P3_f2f3_poly_dim.pkl', 'wb') as file:
#     pickle.dump(new_data, file)

<>:2: DeprecationWarning: invalid escape sequence \P
<>:2: DeprecationWarning: invalid escape sequence \P
<>:2: DeprecationWarning: invalid escape sequence \P
<ipython-input-120-9dea2ec86525>:2: DeprecationWarning: invalid escape sequence \P
  with open('results\P3_f2f3_poly_dim.pkl', 'wb') as file:


In [3]:
#Having completed the sage stuff load back in jupy
with open('results\P3_f2f3_poly_dim.pkl', 'rb') as file:
    data = pickle.load(file)

In [4]:
n=3
number_list = [str(i) for i in range(1, n + 1)] #if n=3 want 123 so can tail ceiling nad det f3(empty) value
max_set  = "".join(number_list)

In [6]:
#outputing examples to text file.

# Open a text file in write mode
with open('results/P3_f2f3_poly_dim.txt', 'w') as file:
    file.write("For f2 in P3 we record for each f2 the set of f3 and the dimension of the polytope of (f2,f3)\n\n")

    for i, sub_list in enumerate(data):
        f2 = sub_list[0]
        file.write(f'f2_{i+1} = {f2}\n')
        packets = sub_list[1:]
        for j,triple in enumerate(packets):
            [ft2,f3,dim]=triple
            
            if f3["empty"]+f3[max_set]==-2:
                file.write(f'{dim}: f3{i+1}{j+1} = {f3}\n')
        file.write("\n")

### P3 translated Done

In [111]:
path="results\poly_inv\P3_trans_f2_f3l.pkl"
n=3

In [112]:
f2f3l=load_l_f2f3(path)
data=f2f3l

In [113]:
#determine all inequalts for f2 f3 and store as  [[f2,[[f2,f3,inequals],...,[f2,f3,inequals]],...,[f2,[[f2,f3,inequals],...,[f2,f3,inequals]]]
out=get_f2f3l_w_inequals(n,data)

In [114]:
# #save to pass to sage
# with open('results\hyperplanes_P3_trans_f2f3.pkl', 'wb') as file:
#     pickle.dump(out, file)

In [113]:
# #load in sage
# with open('results\hyperplanes_P3_trans_f2f3.pkl', 'rb') as file:
#     data = pickle.load(file)

<>:2: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<ipython-input-113-8a799c422c96>:2: DeprecationWarning: invalid escape sequence \h
  with open('results\hyperplanes_P3_f2f3.pkl', 'rb') as file:


In [114]:
n=3
len(data)

10

In [118]:
new_data=get_f2f3l_w_dim_poly(n,data)

In [120]:
# #save from sage
# with open('results\P3_trans_f2f3_poly_dim.pkl', 'wb') as file:
#     pickle.dump(new_data, file)

<>:2: DeprecationWarning: invalid escape sequence \P
<>:2: DeprecationWarning: invalid escape sequence \P
<>:2: DeprecationWarning: invalid escape sequence \P
<ipython-input-120-9dea2ec86525>:2: DeprecationWarning: invalid escape sequence \P
  with open('results\P3_f2f3_poly_dim.pkl', 'wb') as file:


In [7]:
#Having completed the sage stuff load back in jupy
with open('results\P3_trans_f2f3_poly_dim.pkl', 'rb') as file:
    data = pickle.load(file)

In [8]:
n=3
number_list = [str(i) for i in range(1, n + 1)] #if n=3 want 123 so can tail ceiling nad det f3(empty) value
max_set  = "".join(number_list)

In [9]:
#outputing examples to text file.

# Open a text file in write mode
with open('results/P3_trans_f2f3_poly_dim.txt', 'w') as file:
    file.write("For f2 in P3 translated we record for each f2 the set of f3 and the dimension of the polytope of (f2,f3)\n\n")

    for i, sub_list in enumerate(data):
        f2 = sub_list[0]
        file.write(f'f2_{i+1} = {f2}\n')
        packets = sub_list[1:]
        for j,triple in enumerate(packets):
            [ft2,f3,dim]=triple
            
            if f3["empty"]+f3[max_set]==-2:
                file.write(f'{dim}: f3{i+1}{j+1} = {f3}\n')
        file.write("\n")

### P4 Done

In [115]:
path="results\poly_inv\P4_f2_f3l.pkl"
n=4

In [116]:
f2f3l=load_l_f2f3(path)
data=f2f3l

In [117]:
#determine all inequalts for f2 f3 and store as  [[f2,[[f2,f3,inequals],...,[f2,f3,inequals]],...,[f2,[[f2,f3,inequals],...,[f2,f3,inequals]]]
out=get_f2f3l_w_inequals(n,data)

In [118]:
# #save to pass to sage
# with open('results\hyperplanes_P4_f2f3.pkl', 'wb') as file:
#     pickle.dump(out, file)

In [113]:
# #load in sage
# with open('results\hyperplanes_P4_f2f3.pkl', 'rb') as file:
#     data = pickle.load(file)

<>:2: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<ipython-input-113-8a799c422c96>:2: DeprecationWarning: invalid escape sequence \h
  with open('results\hyperplanes_P3_f2f3.pkl', 'rb') as file:


In [136]:
n=4
len(data) #154

In [118]:
new_data=get_f2f3l_w_dim_poly(n,data)

In [119]:
# new_data
"""
    [[{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 1, '123': 1},
      [{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 1, '123': 1},
       {'1': -2,
        '2': -1,
        '3': -1,
        '12': -1,
        '13': -1,
        '23': 0,
        '123': 0,
        'empty': -2},
       'Top dimensional:3'],
      [{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 1, '123': 1},
       {'1': -1,
        '2': -2,
        '3': -1,
        '12': -1,
        '13': 0,
        '23': -1,
        '123': 0,
        'empty': -2},
       'Top dimensional:3'],
      [{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 1, '123': 1},
       {'1': -1,
        '2': -1,
        '3': -1,
        '12': -1,
        '13': -1,
        '23': -1,
        '123': 0,
        'empty': -2},
       'Top dimensional:3']],
     [{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 0, '123': 1},
      [{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 0, '123': 1},
       {'1': -1,
        '2': -2,
        '3': -1,
        '12': -1,
        '13': 0,
        '23': -1,
        '123': 0,
        'empty': -2},
       'Top dimensional:3'],
      [{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 0, '123': 1},
       {'1': -1,
        '2': -1,
        '3': -1,
        '12': -1,
        '13': -1,
        '23': -1,
        '123': 0,
        'empty': -2},
       'Top dimensional:3']],...
"""

[[{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 1, '123': 1},
  [{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 1, '123': 1},
   {'1': -2,
    '2': -1,
    '3': -1,
    '12': -1,
    '13': -1,
    '23': 0,
    '123': 0,
    'empty': -2},
   'Top dimensional:3'],
  [{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 1, '123': 1},
   {'1': -1,
    '2': -2,
    '3': -1,
    '12': -1,
    '13': 0,
    '23': -1,
    '123': 0,
    'empty': -2},
   'Top dimensional:3'],
  [{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 1, '123': 1},
   {'1': -1,
    '2': -1,
    '3': -1,
    '12': -1,
    '13': -1,
    '23': -1,
    '123': 0,
    'empty': -2},
   'Top dimensional:3']],
 [{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 0, '123': 1},
  [{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 0, '123': 1},
   {'1': -1,
    '2': -2,
    '3': -1,
    '12': -1,
    '13': 0,
    '23': -1,
    '123': 0,
    'empty': -2},
   'Top dimensional:3'],
  [{'1': 0, '2': 0, '3': 0, '12': 0, '13': 1, '23': 0, '

In [120]:
# #save from sage
# with open('results\P4_f2f3_poly_dim.pkl', 'wb') as file:
#     pickle.dump(new_data, file)

<>:2: DeprecationWarning: invalid escape sequence \P
<>:2: DeprecationWarning: invalid escape sequence \P
<>:2: DeprecationWarning: invalid escape sequence \P
<ipython-input-120-9dea2ec86525>:2: DeprecationWarning: invalid escape sequence \P
  with open('results\P3_f2f3_poly_dim.pkl', 'wb') as file:


In [10]:
#Having completed the sage stuff load back in jupy
with open('results\P4_f2f3_poly_dim.pkl', 'rb') as file:
    data = pickle.load(file)

In [11]:
n=4
number_list = [str(i) for i in range(1, n + 1)] #if n=3 want 123 so can tail ceiling nad det f3(empty) value
max_set  = "".join(number_list)

In [12]:
#outputing examples to text file.

# Open a text file in write mode
with open('results/P4_f2f3_poly_dim.txt', 'w') as file:
    file.write("For f2 in P4 we record for each f2 the set of f3 and the dimension of the polytope of (f2,f3)\n\n")

    for i, sub_list in enumerate(data):
        f2 = sub_list[0]
        file.write(f'f2_{i+1} = {f2}\n')
        packets = sub_list[1:]
        for j,triple in enumerate(packets):
            [ft2,f3,dim]=triple
            
            if f3["empty"]+f3[max_set]==-2:
                file.write(f'{dim}: f3{i+1}{j+1} = {f3}\n')
        file.write("\n")

### P4 translated Done

In [119]:
path="results\poly_inv\P4_trans_f2_f3l.pkl"
n=4

In [120]:
f2f3l=load_l_f2f3(path)
data=f2f3l

In [121]:
#determine all inequalts for f2 f3 and store as  [[f2,[[f2,f3,inequals],...,[f2,f3,inequals]],...,[f2,[[f2,f3,inequals],...,[f2,f3,inequals]]]
out=get_f2f3l_w_inequals(n,data)

In [122]:
# #save to pass to sage
# with open('results\hyperplanes_P4_trans_f2f3.pkl', 'wb') as file:
#     pickle.dump(out, file)

In [113]:
# #load in sage
# with open('results\hyperplanes_P4_trans_f2f3.pkl', 'rb') as file:
#     data = pickle.load(file)

<>:2: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<ipython-input-113-8a799c422c96>:2: DeprecationWarning: invalid escape sequence \h
  with open('results\hyperplanes_P3_f2f3.pkl', 'rb') as file:


In [114]:
n=4
len(data) #154

10

In [118]:
new_data=get_f2f3l_w_dim_poly(n,data)

In [120]:
# #save from sage
# with open('results\P4_trans_f2f3_poly_dim.pkl', 'wb') as file:
#     pickle.dump(new_data, file)

<>:2: DeprecationWarning: invalid escape sequence \P
<>:2: DeprecationWarning: invalid escape sequence \P
<>:2: DeprecationWarning: invalid escape sequence \P
<ipython-input-120-9dea2ec86525>:2: DeprecationWarning: invalid escape sequence \P
  with open('results\P3_f2f3_poly_dim.pkl', 'wb') as file:


In [13]:
#Having completed the sage stuff load back in jupy
with open('results\P4_trans_f2f3_poly_dim.pkl', 'rb') as file:
    data = pickle.load(file)

In [14]:
n=4
number_list = [str(i) for i in range(1, n + 1)] #if n=3 want 123 so can tail ceiling nad det f3(empty) value
max_set  = "".join(number_list)

In [15]:
#outputing examples to text file.

# Open a text file in write mode
with open('results/P4_trans_f2f3_poly_dim.txt', 'w') as file:
    file.write("For f2 in P4 translated we record for each f2 the set of f3 and the dimension of the polytope of (f2,f3)\n\n")

    for i, sub_list in enumerate(data):
        f2 = sub_list[0]
        file.write(f'f2_{i+1} = {f2}\n')
        packets = sub_list[1:]
        for j,triple in enumerate(packets):
            [ft2,f3,dim]=triple
            
            if f3["empty"]+f3[max_set]==-2:
                file.write(f'{dim}: f3{i+1}{j+1} = {f3}\n')
        file.write("\n")

### P5 Degenerate

In [123]:
path="results\poly_inv\P5_degen_f2_f3l.pkl"
n=5

In [124]:
f2f3l=load_l_f2f3(path)
data=f2f3l

In [125]:
#determine all inequalts for f2 f3 and store as  [[f2,[[f2,f3,inequals],...,[f2,f3,inequals]],...,[f2,[[f2,f3,inequals],...,[f2,f3,inequals]]]
out=get_f2f3l_w_inequals(n,data)

In [126]:
# #save to pass to sage
# with open('results\hyperplanes_P5_degen_f2f3.pkl', 'wb') as file:
#     pickle.dump(out, file)

In [113]:
# #load in sage
# with open('results\hyperplanes_P5_degen_f2f3.pkl', 'rb') as file:
#     data = pickle.load(file)

<>:2: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<ipython-input-113-8a799c422c96>:2: DeprecationWarning: invalid escape sequence \h
  with open('results\hyperplanes_P3_f2f3.pkl', 'rb') as file:


In [114]:
n=5
len(data) #1942??

10

In [118]:
new_data=get_f2f3l_w_dim_poly(n,data)

In [120]:
# #save from sage
# with open('results\P5_degen_f2f3_poly_dim.pkl', 'wb') as file:
#     pickle.dump(new_data, file)

<>:2: DeprecationWarning: invalid escape sequence \P
<>:2: DeprecationWarning: invalid escape sequence \P
<>:2: DeprecationWarning: invalid escape sequence \P
<ipython-input-120-9dea2ec86525>:2: DeprecationWarning: invalid escape sequence \P
  with open('results\P3_f2f3_poly_dim.pkl', 'wb') as file:


In [16]:
#Having completed the sage stuff load back in jupy
with open('results\P5_degen_f2f3_poly_dim.pkl', 'rb') as file:
    data = pickle.load(file)

In [17]:
n=5
number_list = [str(i) for i in range(1, n + 1)] #if n=3 want 123 so can tail ceiling nad det f3(empty) value
max_set  = "".join(number_list)

In [18]:
#outputing examples to text file.

# Open a text file in write mode
with open('results/P5_degen_f2f3_poly_dim.txt', 'w') as file:
    file.write("For f2 in P5 which are degenerate polys, we record for each f2 the set of f3 and the dimension of the polytope of (f2,f3)\n\n")

    for i, sub_list in enumerate(data):
        f2 = sub_list[0]
        file.write(f'f2_{i+1} = {f2}\n')
        packets = sub_list[1:]
        for j,triple in enumerate(packets):
            [ft2,f3,dim]=triple
            
            if f3["empty"]+f3[max_set]==-2:
                file.write(f'{dim}: f3{i+1}{j+1} = {f3}\n')
        file.write("\n")

# MSA case: Investigating whether has empty polytope Pn

## Functions 

In [ ]:
def get_empty_poly(txt):
    """
    Obj:Given list of lists of hyperplanes for each MSA function.
    we construct a list of polytopes RF
    """
    poly_l=[]
    for k in range(0,len(txt)):
        # L_list_line=eval(txt[k])
        L_list_line=txt[k]
        Poly=Polyhedron(ieqs=L_list_line)
        poly_l.append(Poly)

    return poly_l

def check_dim_sizes(empty):

    print("\n","The number of cases where R_f is dim -1 i.e the empty set is:",empty.count(-1))
    print(" The number of cases where R_f is dim 0 is:",empty.count(0))
    print(" The number of cases where R_f is dim 1 is:",empty.count(1))
    print(" The number of cases where R_f is dim 2 is:",empty.count(2))
    print(" The number of cases where R_f is dim 3 is:",empty.count(3))
    print(" The number of cases where R_f is dim 4 is:",empty.count(4))
    print(" The number of cases where R_f is dim 5 is:",empty.count(5))

    print("\n The total number of functions is:",len(empty))
    
def get_formatted_hyperplanes_txt(dataframe): #put into format of sage b<Ax<b+1
    
    """
    If want to do a single function make a df of one funct.
    
    Obj:return a list of equalities to define the polytope for each f in df
    
    In: dataframe of MSA functions
    
    Out:list of list of equality in correct format
    """
    
    #init
    dataframe_list = dataframe.values.tolist()
    df_temp_col_list = dataframe.columns.tolist()
    
    df_col_list =[ list(x) for x in df_temp_col_list]
    
    # print(df_col_list)

    empty_thing=[]#looks like get_formatted_hyperplanes[[[]...[]]...[[]...[]]] , term looks like [[]...[]]
    
    # for i in [dataframe_list[0]]: #creates row at a time
    for i in dataframe_list: #creates row at a time

        l_i= [ [x,] for x in i]

        #Take b<x+y<2
        #looks like [-b,1,1] for x \ge b
        res_lower = [list(itertools.chain(*i)) for i in zip(l_i, df_col_list)] # Ax>b #todo need to insert minus for b term. for Ax-b>0
        for j in range(len(res_lower)):
            
            res_lower[j]=[ int(x) for x in res_lower[j]]#multpliy all elements by negative to change inequality direction.-Ax

            res_lower[j][0]=-int((res_lower[j][0])) # -b part.

        # print("res_lower",res_lower)

        #look like [b+1,-1,-1] for x+y \le b+1
        res_upper = [list(itertools.chain(*i)) for i in zip(l_i, df_col_list)]#Ab<b+1 need to arrange into same format as above -Ax+(b+1)>0
        
        for j in range(len(res_upper)):
            res_upper[j]=[ -1*int(x) for x in res_upper[j]]#multpliy all elements by negative to change inequality direction.-Ax
            
            
            res_upper[j][0]=-int((res_upper[j][0]))+1 # Todo Possible check for negative position

        res_combined= res_lower +res_upper
        
        # print("res_upper",res_upper)

        # empty_thing.append(res_combined) #comment below and run to check both inequalities in b<Ax<b+1

        #change format Dont need sage_format..
        formatted_ResCombine=[]
        for item in res_combined:#res_combined = [[]...[]], item = []
            
            
            x=[0]*(max(len(l) for l in res_combined)) #finding [0,0,0,0,0] for P_4 ie n+1
            x[0]=item[0]
            
            
            for k in range(1,len(item)):
                
                #!!!!
                
                # print("item[k]",item[k])
                
                
                if int(item[k])<0: #!!!!! Changed
                    x[-int(item[k])]=-1 #
                else:
                    x[int(item[k])]=1
            formatted_ResCombine.append(x)

        empty_thing.append(formatted_ResCombine)
        
        
    return empty_thing #a list of the functions P_n

## P3

Here we load the P3 MSA functions and check that all functions RF is top dimensional.

In [90]:
path="results\P3.pkl" #path of hyerplanes...
with open(path, "rb") as g:
    df = pickle.load(g)      
# df.head()

We now obtain for each function the list of hyperplane information in order to pass to sage to build the RF

In [80]:
hyp=get_formatted_hyperplanes_txt(df) #list of lists of hyperplanes for each f in Pn

[['1'], ['2'], ['3'], ['1', '2'], ['1', '3'], ['2', '3'], ['1', '2', '3']]


Because I am stupid and don't know why sage won't install pandas we need to save the hyperplane information so we can avoid using pandas in sage.

In [89]:
# with open('results\hyperplanes_P3.pkl', 'wb') as file:
#     pickle.dump(hyp, file)

In [2]:
with open('results\hyperplanes_P3.pkl', 'rb') as file:
    loaded_list = pickle.load(file)

<>:1: DeprecationWarning: invalid escape sequence \h
<>:1: DeprecationWarning: invalid escape sequence \h
<>:1: DeprecationWarning: invalid escape sequence \h
<ipython-input-2-46c06490b91c>:1: DeprecationWarning: invalid escape sequence \h
  with open('results\hyperplanes_P3.pkl', 'rb') as file:


We now now construct the polytopes

In [14]:
l_polys_p3=get_empty_poly(loaded_list)

In [15]:
dim_l=[]
for p in l_polys_p3:
    dim_l.append(dim(p))

3
3
3
3
3
3
3
3
3
3


In [ ]:
check_dim_sizes(dim_l)

## $P_4$

In [96]:
path="results\P4.pkl" #path of hyerplanes...
with open(path, "rb") as g:
    df = pickle.load(g)      
df.head()

,1,2,3,4,12,13,14,23,24,34,123,124,134,234,1234
0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,2
1,0,0,0,0,0,1,0,1,1,1,1,1,1,2,2
2,0,0,0,0,0,1,0,1,0,1,1,1,1,1,2
3,0,0,0,0,0,1,0,1,1,1,1,1,1,1,2
4,0,0,0,0,0,1,1,1,0,1,1,1,2,1,2


In [97]:
hyp=get_formatted_hyperplanes_txt(df) #list of lists of hyperplanes for each f in Pn

In [98]:
#save
with open('results\hyperplanes_P4.pkl', 'wb') as file:
    pickle.dump(hyp, file)

In [7]:
#load
with open('results\hyperplanes_P4.pkl', 'rb') as file:
    loaded_list = pickle.load(file)

<>:2: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<ipython-input-7-62f03d4d6fde>:2: DeprecationWarning: invalid escape sequence \h
  with open('results\hyperplanes_P4.pkl', 'rb') as file:


In [8]:
l_polys_p4=get_empty_poly(loaded_list)

In [9]:
dim_l=[]
for p in l_polys_p4:
    dim_l.append(dim(p))

In [13]:
check_dim_sizes(dim_l)


 The number of cases where R_f is dim -1 i.e the empty set is: 0
 The number of cases where R_f is dim 0 is: 0
 The number of cases where R_f is dim 1 is: 0
 The number of cases where R_f is dim 2 is: 0
 The number of cases where R_f is dim 3 is: 0
 The number of cases where R_f is dim 4 is: 154
 The number of cases where R_f is dim 5 is: 0

 The total number of functions is: 154


## $P_5$

In [100]:
path="results\P5.pkl" #path of hyerplanes...
with open(path, "rb") as g:
    df = pickle.load(g)      
df.head()

,1,2,3,4,5,12,13,14,15,23,...,234,235,245,345,1234,1235,1245,1345,2345,12345
0,0,0,0,0,0,0,1,1,1,1,...,1,2,2,2,2,2,2,2,2,3
1,0,0,0,0,0,0,1,1,0,1,...,1,1,1,1,2,1,1,2,2,2
2,0,0,0,0,0,0,1,1,0,1,...,1,1,1,2,2,1,2,2,2,2
3,0,0,0,0,0,0,1,1,1,1,...,1,1,1,2,2,2,2,2,2,2
4,0,0,0,0,0,0,1,1,0,1,...,1,1,1,2,2,2,1,2,2,2


In [101]:
hyp=get_formatted_hyperplanes_txt(df) #list of lists of hyperplanes for each f in Pn

In [102]:
#save
with open('results\hyperplanes_P5.pkl', 'wb') as file:
    pickle.dump(hyp, file)

In [14]:
#load
with open('results\hyperplanes_P5.pkl', 'rb') as file:
    loaded_list = pickle.load(file)

<>:2: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<>:2: DeprecationWarning: invalid escape sequence \h
<ipython-input-14-72003e236340>:2: DeprecationWarning: invalid escape sequence \h
  with open('results\hyperplanes_P5.pkl', 'rb') as file:


In [15]:
l_polys_p5=get_empty_poly(loaded_list)

In [16]:
dim_l=[]
for p in l_polys_p5:
    dim_l.append(dim(p))

In [17]:
check_dim_sizes(dim_l)


 The number of cases where R_f is dim -1 i.e the empty set is: 0
 The number of cases where R_f is dim 0 is: 664
 The number of cases where R_f is dim 1 is: 540
 The number of cases where R_f is dim 2 is: 720
 The number of cases where R_f is dim 3 is: 0
 The number of cases where R_f is dim 4 is: 0
 The number of cases where R_f is dim 5 is: 8410

 The total number of functions is: 10334


Previously Alexander Kasprzyk gave the $8410$ count for $P_5$.

We see that,

$$10334=8410+1924$$

And derived the difference between these two cases is as expected.

## Record $f \in P_{5}$ which give empty (<) polytopes

We record he functions with $R_f$ less than top dimensional, these would be empty if Rf was defined with strict inequalities.

In [18]:
def degen_rf_indices(numbers_list):
    result_indices = []
    for i, num in enumerate(numbers_list):
        if num < 5:
            result_indices.append(i)
    return result_indices

In [19]:
p5_degen_rf_indices = degen_rf_indices(dim_l)

In [21]:
# #save
# with open('results\P5_degen_Rf_indices.pkl', 'wb') as file:
#     pickle.dump(p5_degen_rf_indices, file)

<>:2: DeprecationWarning: invalid escape sequence \P
<>:2: DeprecationWarning: invalid escape sequence \P
<>:2: DeprecationWarning: invalid escape sequence \P
<ipython-input-21-8e32de3ccdea>:2: DeprecationWarning: invalid escape sequence \P
  with open('results\P5_degen_Rf_indices.pkl', 'wb') as file:


In [104]:
#load
with open('results\P5_degen_Rf_indices.pkl', 'rb') as file:
    loaded_list = pickle.load(file)

Now take the functions of P5 dataframe with indices in p5_degen_rf_indices

In [106]:
path="results\P5.pkl" #path of hyerplanes...
with open(path, "rb") as g:
    df = pickle.load(g)      
# df.head()

,1,2,3,4,5,12,13,14,15,23,...,234,235,245,345,1234,1235,1245,1345,2345,12345
0,0,0,0,0,0,0,1,1,1,1,...,1,2,2,2,2,2,2,2,2,3
1,0,0,0,0,0,0,1,1,0,1,...,1,1,1,1,2,1,1,2,2,2
2,0,0,0,0,0,0,1,1,0,1,...,1,1,1,2,2,1,2,2,2,2
3,0,0,0,0,0,0,1,1,1,1,...,1,1,1,2,2,2,2,2,2,2
4,0,0,0,0,0,0,1,1,0,1,...,1,1,1,2,2,2,1,2,2,2


In [107]:
# Extract rows based on the indices and create a new DataFrame
new_df = df.iloc[loaded_list]

       1  2  3  4  5  12  13  14  15  23  ...  234  235  245  345  1234  1235  \
16     0  0  0  0  0   0   1   1   1   1  ...    1    1    2    2     2     2   
24     0  0  0  0  0   0   1   1   1   1  ...    1    2    1    2     2     2   
347    0  0  0  0  0   0   1   1   1   1  ...    2    1    2    2     2     2   
354    0  0  0  0  0   0   1   1   1   1  ...    2    2    1    2     2     2   
362    0  0  0  0  0   0   1   1   1   1  ...    1    2    1    2     2     2   
...   .. .. .. .. ..  ..  ..  ..  ..  ..  ...  ...  ...  ...  ...   ...   ...   
10067  0  0  0  0  0   1   0   0   0   0  ...    1    0    0    0     1     1   
10075  0  0  0  0  0   1   0   0   0   0  ...    1    0    0    0     1     1   
10081  0  0  0  0  0   1   0   0   0   0  ...    1    0    0    0     1     1   
10289  0  0  0  0  0   1   0   0   0   0  ...    0    0    1    0     1     1   
10313  0  0  0  0  0   1   0   0   0   0  ...    0    1    0    0     1     1   

       1245  1345  2345  12

In [108]:
#save
with open('results\subset_P5_w_degenRf.pkl', 'wb') as file:
    pickle.dump(new_df, file)

# Used throughout

In [2]:
import numpy as np
import itertools
import pickle

In [3]:
# import pandas as pd

ModuleNotFoundError: No module named 'pandas'

# LO

In [8]:


#now save this and apply analysis back jupyterlab

def inv_empty_selector(data):
    
    """
    In:
    data = [
        [f2, [[f2, f3, poly_dim], ..., [f2, f3, poly_dim]]],...]

    out:

    Obj: Investigate:

    For each item in [[f2,[f2,f3,poly_dim]],...] want to select those where poly_dim="Empty"
    """


    # Function to check if poly_dim is not "Empty"
    def is_not_empty(poly_dim):
        return poly_dim != "Empty"

    # Remove inner lists where poly_dim is not "Empty"
    new_data = [[f2, [inner_list for inner_list in f2_f3_list if is_not_empty(inner_list[2])]] for f2, f2_f3_list in data]

    return new_data

#now we need to display the output

In [ ]:
## $P_2$

In [ ]:
path="\results\P2.pkl"
size=2
empty=get_empty_poly(path,size)

print("\n","The number of cases where R_f is dim -1 i.e the empty set is:",empty.count(-1))
print(" The number of cases where R_f is dim 0 is:",empty.count(0))
print(" The number of cases where R_f is dim 1 is:",empty.count(1))
print(" The number of cases where R_f is dim 2 is:",empty.count(2))

print("\n The total number of functions is:",len(empty))

OSError: [Errno 22] Invalid argument: '\results\\P2.pkl'

In [ ]:
# lbm=np.array([[ 0.,  0.,  1.,  1.],
#        [ 0.,  1.,  0.,  1.],
#        [ 0.,  1.,  1.,  0.],
#        [ 0.,  2.,  0.,  0.],
#        [ 1.,  0.,  0.,  1.],
#        [ 1.,  0.,  1.,  0.],
#        [ 1.,  1.,  0.,  0.],
#        [ 1.,  1.,  1., -1.]])

# # lbm

# phi_G455={"1":1,"2":1.5,"3":1,"4":1.5,"12":1.5,"23":1.5,"34":1.5,"41":1.5}

# print(get_formatted_hyperplanes_txt(lbm,phi_G455))

In [ ]:
# #This is for a specific stability condition on a graph

# def get_formatted_hyperplanes_txt(lbm,phi_G): #put into format of sage b<Ax<b+1
    
#     """
#     Inputs:
#     phi  records indexs of cuts with cutfraction in inequalities.        phi_G62={"1":1,"2":1.5,"3":1,"4":1,"5":1,"6":1.5,"12":1.5,"23":1.5,"34":1,"45":1,"56":1.5,"61":1.5,"123":1.5,"234":1.5,"345":1,"456":1.5,"561":1.5,"612":1}
#     lbm= line bundle mulidegrees given by chip-adding
    
#     Purpose: Get list of inquality data to feed into Sages Polyhedron function
#     https://doc.sagemath.org/html/en/reference/discrete_geometry/sage/geometry/polyhedron/constructor.html
    
    
#     """
    
#     inequals=[]        
#     for d in lbm: #creates row at a time
#         for indexor in phi_G.keys():
            
#             def sum_comps(comp,indexor): #Done
#                 """returns the sum of the components of comp in the position by indexor"""
#                 #for d and phi
#                 sum_l=[comp[int(i)-1] for i in indexor]
#                 return sum(sum_l)
            
            
#             cf=phi_G[indexor] # want value
#             constant_indx= sum_comps(d,indexor)        
        
#             l_i= [ [x,] for x in indexor] #x,y,z.. terms for indexors in Rf space

#             def lower_part(constant_indx,l_i,indexor):
#                 #looks like [-b,1,1] for x \ge b
#                 res_lower = [list(itertools.chain(*i)) for i in zip(l_i, indexor)] # Ax>b #todo need to insert minus for b term. for Ax-b>0

#                 for j in range(len(res_lower)):
#                     res_lower[j][0]=-int((res_lower[j][0])) # -b part.

#                 return res_lower

#             def upper_part(constant_indx,l_i,indexor):

#                 #look like [b+1,-1,-1] for x+y \le b+1
#                 res_upper = [list(itertools.chain(*i)) for i in zip(l_i, indexor)] #Ab<b+1 need to arrange into same format as above -Ax+(b+1)>0

#                 for j in range(len(res_upper)):
#                     res_upper[j]=[ -1*x for x in res_upper[j]] # multpliy all elements by negative to change inequality direction.-Ax
#                     res_upper[j][0]=(-(res_upper[j][0]))+constant_indx 

#                 return res_upper

#             res_lower=lower_part(constant_indx,l_i,indexor)
#             res_upper=upper_part(constant_indx,l_i,indexor)

#             res_combined= res_lower +res_upper

#             # empty_thing.append(res_combined) #comment below and run to check both inequalities in b<Ax<b+1

#             #change format Dont need sage_format..
#             formatted_ResCombine=[]
#             for item in res_combined:#res_combined = [[]...[]], item = []
                
#                 x=[0]*(max(len(l) for l in res_combined)) #finding [0,0,0,0,0] for P_4 ie n+1
#                 x[0]=item[0]
#                 for k in range(1,len(item)):
#                     if item[k]<0:
#                         x[-item[k]]=-1 #
#                     else:
#                         x[item[k]]=1
#                 formatted_ResCombine.append(x)
#             inequals=inequals+formatted_ResCombine
#     return inequals


In [ ]:
#     #Leaving this, this way because im stupid 
#     # for i in values_list: #creates row at a time #Here its of size 1 
#     # print(i)

#     #prep lists with constants

#     # f3 function value
#     l_i= [ [x,] for x in i]
#     # print("l_i",l_i)

#     """
#         Example 
#         Suppose we wish to write 

#         for
#         b<x+y<b+1

#         The inequalities we want are.
#         res_lower_example=[-b,1,1]
#         res_upper_example=[b+1,-1,-1]

#         Outputs:
#         res_lower [[0, 1], [0, 2], [0, 3], [0, 1, 2], [-1, 1, 3], [-1, 2, 3], [-1, 1, 2, 3]]
#         res_upper [[1, -1], [1, -2], [1, -3], [1, -1, -2], [2, -1, -3], [2, -2, -3], [2, -1, -2, -3]]
#     """

#     #looks like [-b,1,1] for b \le x+y
#     # res_lower = [list(itertools.chain(*i)) for i in zip(l_i, df_col_list,df_col_list_comp)] # Ax>b #todo need to insert minus for b term. for Ax-b>0

#     # Combine the lists element-wise using zip
#     combined_lists = zip(l_i, df_col_list, df_col_list_comp)

#     print("combined_lists",list(combined_lists))#combined_lists [([-2], ['1'], ['2', '3']), ([-1], ['2'], ['1', '3']), ([-1], ['3'], ['1', '2']), ([-1], ['1', '2'], ['3']), ([-1], ['1', '3'], ['2']), ([0], ['2', '3'], ['1'])]

#     res_lower=[]
#     for data_list, col_list, col_comp in combined_lists:
#         # Create a list of lists for coefficients with respective signs

#         # Given inputs
#         # data_list  = [-2]
#         # col_list = ['1']
#         formated_col_l=[int(x) for x in col_list]
#         # col_comp_list = ['2', '3']
#         form_col_comp_list=[-int(x) for x in col_comp_list]
#         # Create the result list using list comprehension
#         res_lower_term=data_list + formated_col_l+form_col_comp_list
#         # print(result)  # Output: [-2, 1, -2, -3]

#         res_lower.append(res_lower_term)

#     print(f"res_lower[0]:{res_lower[0]}") #res_lower[0]:[-2, '1', '2', '3']





# #         for j in range(len(res_lower)):
# #             res_lower[j]=[ int(x) for x in res_lower[j]]#multpliy all elements by negative to change inequality direction.-Ax
# #             res_lower[j][0]=-int((res_lower[j][0])) # -b part.
# #         # print("res_lower",res_lower) 

#     #--------------------------------------------


#     res_upper = [list(itertools.chain(*i)) for i in zip(l_i, df_col_list,df_col_list_comp)]#Ab<b+1 need to arrange into same format as above -Ax+(b+1)>0
#     for j in range(len(res_upper)):
#         res_upper[j]=[ -1*int(x) for x in res_upper[j]]#multpliy all elements by negative to change inequality direction.-Ax
#         res_upper[j][0]=-int((res_upper[j][0]))+1 # Todo Possible check for negative position
#     # print("res_upper",res_upper) #look like [b+1,-1,-1] for x+y \le b+1
    
    
    
#     """
#             # dataframe = pd.DataFrame([f3_copy])
#             # f3={'1': -2, '2': -1, '3': -1, '12': -1, '13': -1, '23': 0}
#         # print(f3)

#         # dataframe_list = dataframe.values.tolist()
#         # print(dataframe_list)#[[-2,-1,-1,-1,-1,0]]

#         # df_temp_col_list = dataframe.columns.tolist()
#         # print(df_temp_col_list)#['1', '2', '3', '12', '13', '23']
#             #[['1'], ['2'], ['3'], ['1', '2'], ['1', '3'], ['2', '3']]
#         # print("df_col_list_comp:",df_col_list_comp) #df_col_list_comp: [['2', '3'], ['1', '3'], ['1', '2'], ['3'], ['2'], ['1']]
#     """
#     """If
    
#         dataframe_list=[[-2,-1,-1,-1,-1,0]]

#         df_col_list =[['1'], ['2'], ['3'], ['1', '2'], ['1', '3'], ['2', '3']]

#         df_col_list_comp= [['2', '3'], ['1', '3'], ['1', '2'], ['3'], ['2'], ['1']]

#         what is

#         res_lower = [list(itertools.chain(*i)) for i in zip(l_i, df_col_list,df_col_list_comp)] # Ax>b #todo need to insert minus for b term. for Ax-b>0
#     """
    